## COVID - Ocupação Hospitalar (2020 ~ 2022) 
#### - - - - *em progresso* - - - -

Utilizando dados fornecidos pelo Ministério da Saúde, iremos analisar as **variações na ocupação hospitalar no país, com *foco em ocorrências de COVID-19*, para os anos 2020, 2021 e 2022.**
Os conjuntos de dados para cada ano podem ser baixados no formato `.csv` através deste [link](https://dados.gov.br/dados/conjuntos-dados/registro-de-ocupacao-hospitalar-covid-19). O link redireciona para uma página específica dentro do **[Portal Brasileiro de Dados Abertos](https://dados.gov.br/home)**

No entanto, **não há documentação disponível**. As únicas informações disponibilizadas a respeito dos dados são:
>"O módulo Internações foi desenvolvido para registro da ocupação de leitos clínicos e de Unidade de Terapia Intensiva (UTI) SUS destinados para atendimento aos pacientes com casos suspeitos ou confirmados da COVID-19 (ocupação SRAG / COVID-19)"

e também:

>"A partir do ano de 2022 foram acrescentado novos campos utilizados para descrever a ocupação dos leitos. Registros com ano anterior a 2022 não contém esses campos preenchidos."

Com essa base, vamos explorar o conjunto de dados para entender melhor o que temos em mãos.

In [1]:
import pandas as pd
import utils

Em um primeiro momento, vamos analisar os dados de 2020.
> Dados obtidos em 17/04/2023

## Data Wrangling

In [2]:
df = pd.read_csv('ocupacao-leito-2020.csv')

/tmp/ipykernel_9637/2661094983.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ocupacao-leito-2020.csv')


Já somos recebidos com um aviso indicando que há uma coluna com tipos de dados misturados, mas vamos além para investigar o restante e conhecer melhor nosso material de análise.

Para isso será útil verificar uma amostra do conteúdo do conjunto de dados e listar todas as colunas junto com seus respectivos tipos de dados. Faremos isso com o método `.head()` e atributo `dtypes` respectivamente.



In [3]:
df.head()

,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
0,0,p5Ez41Zu6B,2020-04-14T03:00:00.000Z,2303167,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$auVMjEVEVz,Santa Catarina,Itapema,Santa Catarina,Itapema,True,False,2020-04-15T13:44:30.085Z,2020-04-16T13:56:13.656Z
1,1,eUOMsSt7T7,2020-04-15T03:00:00.000Z,5935377\t,1.0,0.0,0.0,0.0,NaN,NaN,...,parse-cloud,_User$WBa5nfe9P9,Rio de Janeiro,Rio de Janeiro,Rio de Janeiro,Rio de Janeiro,False,False,2020-04-15T13:50:47.256Z,2020-08-01T20:08:13.116Z
2,2,QaG4oIBrMa,2020-04-15T14:57:16.255Z,0012599,15.0,1.0,3.0,2.0,NaN,NaN,...,parse-cloud,_User$6oxZQHBPQ7,Rio de Janeiro,Niterói,Rio de Janeiro,Niterói,False,False,2020-04-15T15:20:57.033Z,2020-08-01T20:02:12.729Z
3,3,ifya31F3ZF,2020-03-24T03:00:00.000Z,2562871,9.0,0.0,1.0,0.0,NaN,NaN,...,parse-cloud,_User$fhufIbmR9C,Ceará,Várzea Alegre,Ceará,Várzea Alegre,False,False,2020-04-15T16:58:32.839Z,2020-08-01T21:40:37.624Z
4,4,dkurDhelx9,2020-04-01T03:00:00.000Z,2252228,26.0,0.0,1.0,0.0,NaN,NaN,...,parse-cloud,_User$tkAdHTNhC9,Rio Grande do Sul,Encantado,Rio Grande do Sul,Encantado,False,False,2020-04-15T17:02:53.877Z,2020-08-01T21:37:53.664Z


In [4]:
df.dtypes

Unnamed: 0                 int64
_id                       object
dataNotificacao           object
cnes                      object
ocupacaoSuspeitoCli      float64
ocupacaoSuspeitoUti      float64
ocupacaoConfirmadoCli    float64
ocupacaoConfirmadoUti    float64
ocupacaoCovidUti         float64
ocupacaoCovidCli         float64
ocupacaoHospitalarUti    float64
ocupacaoHospitalarCli    float64
saidaSuspeitaObitos      float64
saidaSuspeitaAltas       float64
saidaConfirmadaObitos    float64
saidaConfirmadaAltas     float64
origem                    object
_p_usuario                object
estadoNotificacao         object
municipioNotificacao      object
estado                    object
municipio                 object
excluido                    bool
validado                    bool
_created_at               object
_updated_at               object
dtype: object

A coluna sobre a qual recebemos o aviso se trata de `cnes`, referente ao *Cadastro Nacional de Estabelecimentos de Saúde*. Pelo que é possível verificar, deveriamos ver apenas números, mas aparentemente existem erros de digitação inserindo caractéres em alguns registros.

Além disso, é possível verificar alguns campos iniciados com "**_**", que geralmente indicam *variáveis privadas voltadas para uso interno de alguma aplicação*. **Possivelmente não acrescentarão muito para a análise**, mas é importante verificar se de fato podemos excluí-los sem perdas.

Analisando os primeiros registros é possível notar que `_created_at` parece ser a data em que o registro foi criado. Muitos deles são feitos em sequência num *curto período de tempo*, adicionando registros `dataNotificacao` de datas passadas. 

`_updated_at` apresenta padrão semelhante (inserção em lotes), provavelmente gerando uma análise desconexa. Logo, `dataNotificacao` seria a infomação efetivamente útil para nossa análise. 

`_p_usuario` proavelmente se refere ao 'id' do usuário que inseriu o registro, enquanto `_id` possivelmente se trata do identificador do registro em si. Podemos verificar se a quantidade de `_id` únicos é igual ao número total de registros.

In [5]:
shape = df.shape
shape[0] == len(df['_id'].unique())

True

`_id` é mesmo o que imaginamos. A propósito, podemos aproveitar e conferir com quantos registros estamos lidando:

In [6]:
print(f"Nosso DataFrame possui {shape[0]} linhas e {shape[1]} colunas")

Nosso DataFrame possui 554687 linhas e 26 colunas


Vamos começar a listar todas as colunas que não terão utilidade para a análise.

In [7]:
to_drop = ['_id', '_p_usuario', '_created_at', '_updated_at']

Agora podemos prosseguir para as demais colunas. 

Logo de início podemos notar que nas 5 primeiras linhas, todos os registros de `ocupacaoCovidUti` e `ocupacaoCovidCli` não estão preenchidos. Será que isso se mantém para todo o documento?

In [8]:
df['ocupacaoCovidUti'].value_counts()

ocupacaoCovidUti
 0.0     336
 20.0     67
 19.0     41
 1.0      21
 18.0     16
 2.0       5
 3.0       4
 17.0      3
 10.0      3
 6.0       3
 7.0       2
 5.0       2
 4.0       1
-1.0       1
 8.0       1
 44.0      1
 22.0      1
 25.0      1
 23.0      1
 11.0      1
Name: count, dtype: int64

Dos mais de 550 mil registros no documento, na coluna `ocupacaoCovidUti` somente cerca de 500 possuem um valor válido (incluindo um valor negativo).

Por padrão, o método `.head()` não mostra todas as colunas, então é possível que além de `ocupacaoCovidUti` e `ocupacaoCovidCli` outras colunas apresentem o mesmo padrão. 

Mesmo que este arquivo seja referente ao ano 2020, vimos que existe um campo específico para data de *atualização do registro*. Além disso, segundo a fonte, *registros a partir de 2022 possuem novos campos*. Dessa maneira, **é possível que dados atualizados depois de 2022 apresentem inconsistências**.

Podemos começar a investigar essa hipótese verificando melhor as estatísticas descritivas dos campos numéricos.

In [9]:
df.describe()

,Unnamed: 0,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,ocupacaoHospitalarUti,ocupacaoHospitalarCli,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
count,5.546870e+05,5.542860e+05,554286.000000,554287.000000,554287.000000,511.000000,511.000000,511.000000,511.000000,554571.000000,5.545830e+05,554675.000000,5.546740e+05
mean,2.802098e+05,1.901152e+02,7.139670,6.081243,4.477320,5.301370,4.651663,7.043053,22.080235,0.224539,5.138644e+01,0.373017,3.395598e+01
std,1.677398e+05,6.208808e+04,16.098472,33.178988,11.254881,8.597718,5.848411,12.397110,51.228089,3.571056,3.766872e+04,3.744652,1.729309e+04
min,0.000000e+00,-5.700000e+01,-9.000000,-10.000000,-7.000000,-1.000000,0.000000,0.000000,0.000000,-10.000000,-1.200000e+01,-3.000000,-1.300000e+01
25%,1.387045e+05,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,2.773850e+05,4.000000e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,8.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
75%,4.160795e+05,2.700000e+01,8.000000,6.000000,5.000000,17.500000,10.000000,20.000000,11.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
max,1.618772e+06,2.908202e+07,924.000000,22222.000000,334.000000,44.000000,60.000000,195.000000,314.000000,425.000000,2.805202e+07,322.000000,9.112020e+06


Constatamos exatamente 511 registros nas colunas `ocupacaoCovidUti`, `ocupacaoCovidCli`, `ocupacaoHospitalarUti`, `ocupacaoHospitalarCli`, o que corrobora para nossa hipótese de registros atualizados depois de 2022.

Para confirmar, podemos averiguar as datas de atualização dos registros de qualquer um dos 4 campos em questão.

In [10]:
df.loc[df['ocupacaoCovidUti'].notnull(), '_updated_at']

14327     2022-02-21T17:57:08.292Z
55820     2022-02-21T18:02:56.533Z
103291    2021-10-05T13:25:13.346Z
104964    2021-10-05T13:26:12.193Z
113207    2021-10-05T13:26:45.066Z
                    ...           
554682    2023-02-15T19:28:39.477Z
554683    2023-03-08T12:03:10.135Z
554684    2023-04-06T14:28:58.666Z
554685    2023-04-12T12:57:54.509Z
554686    2023-04-12T12:58:12.391Z
Name: _updated_at, Length: 511, dtype: object

Considerando os 511 registros contestados, a atualização mais antiga ocorreu em 21/02/**2022**, e a mais recente em 12/04/**2023**, comprovando nossa hipótese.

Como não temos o conhecimento exato da informação que cada campo nos traz, a julgar pelo nome das colunas, parecem se tratar de *informações objetivamente diferentes*. Ainda, suas estatísticas não parecem diferir muito do resto do conjunto. Com essas considerações, *junto com o fato de representarem cerca de 0.001% dos dados*, acredito que seja melhor não incluí-las na análise.

In [11]:
for item in ['ocupacaoCovidUti', 'ocupacaoCovidCli', 'ocupacaoHospitalarUti', 'ocupacaoHospitalarCli']:
    to_drop.append(item)

O campo `origem` aparentemente indica a forma com que os dados foram coletados ou processados. Para ter mais certeza, podemos verificar quais os valores encontrados.

In [12]:
df['origem'].unique()

array(['aplicacao-web', 'parse-cloud', 'RPA-SP', 'RPA-CE', 'RPA-RS',
       'RPA-MS', 'RPA-GO', 'RPA-MT', 'RPA-RN', 'RPA-PR', 'RPA-MG',
       'RPA-MG-BELO-HORIZONTE', 'RPA-PR-CURITIBA'], dtype=object)

"RPA" provavelmente se refere à "Robotic Process Automation", enquanto "parse-cloud" pode se referir a um serviço de nuvem que faz a análise sintática (parsing) dos dados. De toda maneira, não é algo que será levado em consideração na análise de ocupação hospitalar.

In [13]:
to_drop.append('origem')

Veremos agora o que `excluido` e `validado` podem nos mostrar.

In [14]:
df['excluido'].value_counts()

excluido
False    554182
True        505
Name: count, dtype: int64

In [15]:
df.loc[df['excluido'] == True]


,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
0,0,p5Ez41Zu6B,2020-04-14T03:00:00.000Z,2303167,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$auVMjEVEVz,Santa Catarina,Itapema,Santa Catarina,Itapema,True,False,2020-04-15T13:44:30.085Z,2020-04-16T13:56:13.656Z
31,31,tv1uXbMbdc,2020-04-15T17:23:44.000Z,2240335,20.0,5.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$Enseqp32PQ,Rio Grande do Sul,Farroupilha,Rio Grande do Sul,Farroupilha,True,False,2020-04-15T17:24:00.316Z,2020-04-15T17:35:07.999Z
62,62,fHqEA6b8Xt,2020-04-15T19:18:27.505Z,2375850,0.0,0.0,0.0,10.0,NaN,NaN,...,aplicacao-web,_User$Utr4cDP5eD,Mato Grosso do Sul,Paranaíba,Mato Grosso do Sul,Paranaíba,True,False,2020-04-15T19:29:13.561Z,2020-04-15T19:48:07.806Z
70,70,Zekk3kC7Gp,2020-04-01T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:38:08.434Z,2020-04-15T19:49:49.157Z
72,72,6m0iH5blyQ,2020-04-02T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:39:52.298Z,2020-04-15T19:50:00.013Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19617,19621,JO81FFRtz8,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:20:38.299Z,2020-05-12T11:09:21.900Z
19618,19622,EACElG3PDS,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:21:46.909Z,2020-05-12T11:09:30.965Z
19619,19623,S63bzGI2iK,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:20:47.607Z,2020-05-12T11:10:07.500Z
19813,19821,OZzcvODn1G,2020-05-11T15:38:05.414Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T15:40:22.575Z,2020-05-12T11:09:17.290Z


Apesar de não haver documentação disponível, o nome da coluna em si (`excluido`) já indica fortemente que os registros marcados como verdadeiros não são aptos para uso nessa análise. 

Encontramos **505 registros** marcados, e destes podemos notar algumas particularidades como: 

Registros `dataNotificacao` exatamente iguais *(com precisão de 0.001 segundos)* para a mesma `cnes` e feitos pelo mesmo `_p_usuario`, vários deles com dados idênticos em `ocupacaoSuspeitoCli`, `ocupacaoSuspeitoUti`, `ocupacaoConfirmadoCli`, `ocupacaoConfirmadoUti`.

In [16]:
df.loc[(df['excluido'] == True) & (df.duplicated(subset=['dataNotificacao']))]

,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
70,70,Zekk3kC7Gp,2020-04-01T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:38:08.434Z,2020-04-15T19:49:49.157Z
72,72,6m0iH5blyQ,2020-04-02T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:39:52.298Z,2020-04-15T19:50:00.013Z
73,73,J6qrjnb2y4,2020-04-03T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:41:13.629Z,2020-04-15T19:50:20.836Z
74,74,84u09qrCH4,2020-04-04T03:00:00.000Z,2760819,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:41:51.378Z,2020-04-15T19:50:07.477Z
186,186,lMXoydTmd1,2020-04-14T03:00:00.000Z,2418967,25.0,0.0,2.0,0.0,NaN,NaN,...,aplicacao-web,_User$1aBS46RcA0,Santa Catarina,São João Batista,Santa Catarina,São João Batista,True,False,2020-04-15T20:24:50.502Z,2020-04-15T21:42:10.108Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13772,13776,deIHZTOWsU,2020-04-02T03:00:00.000Z,0105724,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$QtAd4NBwBc,Paraná,Corbélia,Paraná,Corbélia,True,False,2020-05-04T20:25:54.924Z,2020-05-04T20:26:32.391Z
18500,18504,8tidqNgXtv,2020-05-09T17:30:23.940Z,2436450,117.0,20.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-09T18:34:15.464Z,2020-05-12T11:09:58.154Z
19618,19622,EACElG3PDS,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:21:46.909Z,2020-05-12T11:09:30.965Z
19619,19623,S63bzGI2iK,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:20:47.607Z,2020-05-12T11:10:07.500Z


Muitos registros de fato seguem o padrão que identificamos. Provavelmente são registros desatualizados ou com algum tipo de inconsistência.

Além do nome sugestivo da coluna, obtivemos várias informações que nos permitem ser **assertivos na eliminação de todos os registros em que `excluido` = `True`**. Como após a exclusão a coluna não terá mais serventia, podemos também eliminá-la dos dados para análise.

In [17]:
df.drop(df.loc[df['excluido'] == True].index, inplace=True)

In [18]:
to_drop.append('excluido')

Agora vejamos `validado`:

In [19]:
df['validado'].value_counts()

validado
False    554182
Name: count, dtype: int64

Todos os registros estão marcados como `False`, o que nos impossibilita buscar por padrões para inferir o que de fato a coluna significa.

Em busca de mais informações, encontrei outro [local onde o mesmo conjunto de dados é disponibilizado](https://opendatasus.saude.gov.br/dataset/registro-de-ocupacao-hospitalar-covid-19), o *OpenDataSUS*. Verifiquei que tanto o [Portal Brasileiro de Dados Abertos](https://dados.gov.br/home) *(onde obtivemos os dados)* quanto o [OpenDataSUS](https://opendatasus.saude.gov.br/) **apontam para o mesmo [URL](https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/LEITOS/2023-04-21/esus-vepi.LeitoOcupacao_2020.csv)** 
> *(o arquivo está armazenado em um bucket S3 da Amazon. Clicar no link 'URL' inicia o download)*. 

Ou seja, tanto um quanto outro nos fornecem o **mesmo arquivo**. Logo, qualquer informação encontrada em ambos endereços é igualmente válida para os dados obtidos.

No OpenDataSUS temos algumas [informações adicionais](https://opendatasus.saude.gov.br/dataset/registro-de-ocupacao-hospitalar-covid-19/resource/a16c8d5f-dc6d-46c8-8e75-41765a18a289) sobre o conjunto de dados em si. Temos informações como data de criação, **status de atividade** e **data da última modificação** *(a última presente em ambos)*. 

* Estado: ativo
* Dados atualizados pela última vez: 22/04/2023 *(dia em que foi feita a verificação)*

Também podemos encontrar registros do [fluxo de atividade](https://opendatasus.saude.gov.br/dataset/activity/registro-de-ocupacao-hospitalar-covid-19). Aqui vemos que os dados são **atualizados diariamente**.

Sabendo que se trata de um arquivo atualizado e mantido sob cuidado diário, podemos concluir que os dados em que estamos trabalhando são utilizáveis, ainda que o campo `validado` possa nos induzir a supor o contrário.

Dessa maneira, como todos os registros do arquivo possuem o mesmo valor, nada é agregado à análise. Iremos apenas *descartar a coluna*.

In [20]:
to_drop.append('validado')

Ainda nos restam as colunas `municipio`, `estado`, `municipioNotificacao`, `estadoNotificacao`. Vamos apenas conferir se os "pares" (ex: `estado` / `estadoNotificacao`) contém o mesmo conteúdo.

In [21]:
len(df['estado'].unique()) == len(df['estadoNotificacao'].unique())

False

In [22]:
len(df['estado']) == len(df['estadoNotificacao'])

True

Há algo de diferente aqui.

Apesar de possuirem o mesmo número de registros, temos diferentes quantidade de valores únicos.

In [23]:
print(df['estado'].unique())

['Rio de Janeiro' 'Ceará' 'Rio Grande do Sul' 'Santa Catarina'
 'Minas Gerais' 'Pará' 'Piauí' 'Mato Grosso do Sul' 'Goiás' 'Rondônia'
 'Acre' 'Amazonas' 'Pernambuco' 'Paraná' 'Maranhão' 'Bahia' 'São Paulo'
 'Paraíba' 'Alagoas' 'Mato Grosso' 'Sergipe' 'Distrito Federal'
 'Tocantins' 'Rio Grande do Norte' nan 'Amapá' 'Espírito Santo' 'Roraima'
 'GOIAS']


In [24]:
print(df['estadoNotificacao'].unique())

['Rio de Janeiro' 'Ceará' 'Rio Grande do Sul' 'Santa Catarina'
 'Minas Gerais' 'Pará' 'Piauí' 'Mato Grosso do Sul' 'Goiás' 'Rondônia'
 'Acre' 'Amazonas' 'Pernambuco' 'Paraná' 'Maranhão' 'Bahia' 'São Paulo'
 'Paraíba' 'Alagoas' 'Mato Grosso' 'Sergipe' 'Distrito Federal'
 'Tocantins' 'Rio Grande do Norte' 'Amapá' 'Espírito Santo' 'Roraima'
 'GOIAS']


Em ambos o estado de Goiás está registrado duas vezes. Como "Goiás" e "GOIAS". Isso será fácil de corrigir.

Mas vemos algo além. `estado` possui alguns campos listados como `nan` *("not a number" nesse contexto é referente a um campo nulo)*. Vamos verificar quais são esses registros.

In [25]:
df[df['estado'].isnull()]

,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
4182,4184,bc66V3say8,2020-04-24T12:32:29.807Z,2246902,1.0,0.0,1.0,0.0,NaN,NaN,...,aplicacao-web,_User$Ow1jWbSgxK,Rio Grande do Sul,Não-Me-Toque,NaN,NaN,False,False,2020-04-24T12:33:53.952Z,2020-04-24T12:33:53.952Z


Apena um registro tem campos faltantes em `estado` e `municipio`. Porém seus "pares" contam esses dados.

Poderiamos dar esse caso em particular como resolvido, no entanto, será que todos os "pares" apresentam as mesmas informações?

In [26]:
len(df.loc[df['estado'] == df['estadoNotificacao']]) == df.shape[0]

False

Aperentemente não. Vejamos o motivo:

In [27]:
df.loc[df['estado'] != df['estadoNotificacao'], ['cnes', 'municipio', 'estado' , 'municipioNotificacao', 'estadoNotificacao']]

,cnes,municipio,estado,municipioNotificacao,estadoNotificacao
931,2438313,Minaçu,Goiás,Itaobim,Minas Gerais
2962,703208645495691,Buritis,Rondônia,São Romão,Minas Gerais
4182,2246902,NaN,NaN,Não-Me-Toque,Rio Grande do Sul
33764,223505,Cassilândia,Mato Grosso do Sul,Ouro Preto do Oeste,Rondônia
121811,2786095.0,Dianópolis,Tocantins,Brasília,Distrito Federal
193605,4069803,Juína,Mato Grosso,Propriá,Sergipe
438983,2013029,Manaus,Amazonas,Tunápolis,Santa Catarina
478919,3004104,Manaus,Amazonas,Propriá,Sergipe
516046,2017555,Carauari,Amazonas,Codó,Maranhão


Vemos algo sem coerência. Poderíamos discutir sobre o significado de cada coluna. Porém, dificilmente teríamos certeza do que se tratam *(ex: residente de um estado sendo notificado em outro durante uma viagem)*. 

Dessa forma, será melhor excluir os registros da análise. Também poderíamos argumentar que 9 registros são uma parcela muito pequena do arquivo.

In [28]:
print(f"9 registros representam {(9 / df.shape[0]):.6f}% do arquivo")

9 registros representam 0.000016% do arquivo


In [29]:
df.drop(df.loc[df['estado'] != df['estadoNotificacao']].index, inplace=True)

len(df.loc[df['estado'] == df['estadoNotificacao']]) == df.shape[0]

True

Agora `estado` e `estadoNotificacao` são idênticas, podemos utilizar tanto uma quanto outra. Poderíamos executar o mesmo processo para `municipio` e `municipioNotificacao`, mas vamos fazer outra consideração antes.

**Qual o escopo da análise?**

A idéia principal é de analisar o nível de ocupação hospitalar **em nível nacional**, portanto, talvez não seja interessante aumentarmos a "resolução" para municípios. Incorremos em obscurecer nuances pelo exagero de detalhes.

O que nos leva a considerar quais colunas são mais apropriadas e quais são menos compatíveis com o objetivo da análise.

Pelo menos em um primeiro momento, as informações mais relevantes seriam `estado`, `dataNotificacao`, `ocupcacaoSuspeitoCli`, `ocupacaoSuspeitoUti`, `ocupacaoConfirmadoCli`, `ocupacaoConfirmadoUti`, `saidaSuspeitaAltas`, `saidaSuspeitaObitos`, `saidaConfirmadaAltas`, `saidaConfirmadaObitos`. 

Campos como `municipio` e `cnes` entregam detalhes fora do escopo desta análise. Porém, são bons dados para análises subsequentes. *Por exemplo, após verificarmos os estados mais e menos afetados, poderíamos examiná-los com mais detalhes.*

In [30]:
print(f"No momento, o tamanho do nosso DataFrame é {(df.memory_usage().sum() / 10**6):.1f} Mb")

No momento, o tamanho do nosso DataFrame é 111.9 Mb


Temos algumas opções para separar os dados sobre municípios, incluindo bancos de dados relacionais para poupar armazenamento. 

No atual estado, o DataFrame tem aproximadamente 112 Mb. Não é um arquivo grande, mas de toda maneira, após a remoção das colunas marcadas o tamanho será consideravelmente reduzido. Vamos fazer isso para considerar nossas opções.

Por último vamos apenas adicionar `estadoNotificacao` em nossa lista. *Como ainda não vasculhamos os municípios, vamos deixá-los como estão por enquanto*

In [31]:
to_drop.append('estadoNotificacao')
print(to_drop)

['_id', '_p_usuario', '_created_at', '_updated_at', 'ocupacaoCovidUti', 'ocupacaoCovidCli', 'ocupacaoHospitalarUti', 'ocupacaoHospitalarCli', 'origem', 'excluido', 'validado', 'estadoNotificacao']


E finalmente vamos excluir todas as colunas que selecionamos

In [32]:
df.drop(columns=to_drop, inplace=True)
print(f"Agora nosso DataFrame tem {(df.memory_usage().sum() / 10**6):.1f} Mb")

Agora nosso DataFrame tem 66.5 Mb


Quase metade do tamanho original. E ainda não eliminamos as colunas relacionadas a municípios.

Podemos fazer isso sem prejuízo, mas antes, como podemos utilizar esses dados em outra análises, vamos salvar o DataFrame como um novo arquivo.

Considerando que temos um arquivo pequeno e não estamos reduzindo espaço de armazenamento a todo custo, transformar nossos dados atuais em tabelas em um banco de dados relacional pode não ser a solução mais prática. No lugar disso podemos apenas salvar o estado atual do DataFrame em um novo arquivo `.csv`.

Como não haverá necessidade de uma chave primária para cada registro, podemos também deletar a coluna `Unnamed: 0` antes de criar o novo arquivo.

In [33]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.to_csv('ocupacao-leito-2020-municipios.csv', index=False, encoding='utf-8')

Com isso feito, podemos retirar as colunas restantes.

In [35]:
df.drop(columns=['cnes', 'municipio', 'municipioNotificacao'], inplace=True)

Apenas por curiosidade, podemos checar o quanto reduzimos o arquivo.

In [36]:
print(f"Agora nosso DataFrame tem {(df.memory_usage().sum() / 10**6):.1f} Mb")

Agora nosso DataFrame tem 48.8 Mb
